In [1]:
# Reference: https://www.kaggle.com/dansbecker/transfer-learning
# importing the libraries
import numpy as np
import pandas as pd
import matplotlib
import os
import cv2
import random
import keras

from glob import glob
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from keras.applications.resnet50 import ResNet50
import tensorflow as tf
from keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [10]:
# loading the directories 
training_dir = 'C:/Users/avadh/Desktop/CMPE 295A/Dataset/Temp/Semi_Train'
test_dir = 'C:/Users/avadh/Desktop/CMPE 295A/Dataset/Temp/Semi_Test'
folders=os.listdir("C:/Users/avadh/Desktop/CMPE 295A/Dataset/Temp/Semi_Train")
print(folders)
#test_dir = '../input/fruits/fruits-360_dataset/fruits-360/test-multiple_fruits/'

['Chiffon', 'Jeans', 'Lace_Dress', "Men's_Shirts", 'Skirt']


In [11]:
size=64,64

In [12]:
# useful for getting number of files
image_files = glob(training_dir + '/*/*.jp*g')
test_image_files = glob(test_dir + '/*/*.jp*g')

In [13]:
image_train=[]
image_labels=[]
image_names=[]

for folder in folders:
    for each in os.listdir(os.path.join(training_dir,folder)):
        if each.endswith('jpg'):
            image_names.append(os.path.join(training_dir,folder,each))
            image_labels.append(folder)
            img=cv2.imread(os.path.join(training_dir,folder,each))
            img_in=cv2.resize(img,size)
            image_train.append(img_in)
        else:
            continue


In [14]:
union_list=list(zip(image_train,image_labels))
random.shuffle(union_list)
train,labels=zip(*union_list)
X=np.array(train)
Y=np.array(labels)

In [15]:
X=X/255.0


In [16]:
le=LabelEncoder()
Y=le.fit_transform(Y)
Y=to_categorical(Y,num_classes=len(folders))

In [17]:
X_train,X_val,Y_train,Y_val=train_test_split(X,Y,test_size=0.1)


In [18]:
resnet_model=ResNet50(include_top=False,input_tensor=keras.engine.input_layer.Input(shape=(64, 64, 3)))

C:\Users\avadh\Anaconda3\envs\tensorflow\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [19]:
model=Sequential()
model.add(resnet_model)
model.add(Flatten())
model.add(Dense(4096,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096,activation='relu'))
model.add(Dropout(0.5))

In [20]:
model.add(Dense(len(folders),activation='softmax'))

model.layers[0].trainable = False

In [21]:
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2, 2, 2048)        23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              33558528  
_________________________________________________________________
dropout_1 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 4096)              16781312  
_________________________________________________________________
dropout_2 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 20485     
Total para

In [22]:
optimizer=RMSprop(lr=0.0001)   #,rho=0.9,epsilon=1e-08,decay=0.0)
model.compile(optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
learning_rate_reduction=ReduceLROnPlateau(monitor='val_acc',patience=3,verbose=1,factor=0.5,minlr=0.00001)

In [23]:
datagen = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False,  
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        zca_whitening=False,
        rotation_range=10,  
        zoom_range = 0.1, 
        width_shift_range=0.1,
        height_shift_range=0.1,  
        horizontal_flip=False,
        vertical_flip=False) 

datagen.fit(X_train)

In [24]:
history=model.fit_generator(datagen.flow(X_train,Y_train,batch_size=10),epochs=5,validation_data=(X_val,Y_val),
                          verbose=1,steps_per_epoch=X_train.shape[0]/10)


Epoch 1/5
536/535 [==============================] - 1450s 3s/step - loss: 1.8620 - acc: 0.4732 - val_loss: 1.8263 - val_acc: 0.3742
Epoch 2/5
536/535 [==============================] - 1109s 2s/step - loss: 1.4475 - acc: 0.5534 - val_loss: 1.9109 - val_acc: 0.3742
Epoch 3/5
536/535 [==============================] - 1161s 2s/step - loss: 1.2946 - acc: 0.5863 - val_loss: 2.2104 - val_acc: 0.3054
Epoch 4/5
536/535 [==============================] - 1516s 3s/step - loss: 1.2380 - acc: 0.6073 - val_loss: 1.8124 - val_acc: 0.3054
Epoch 5/5
536/535 [==============================] - 1529s 3s/step - loss: 1.1714 - acc: 0.6153 - val_loss: 1.8996 - val_acc: 0.3070


In [25]:
from keras.models import model_from_json

# serialize model to JSON
model_json = model.to_json()
with open("Dataset/MyVersion/ResNet50.json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk
